In [1]:
import kfp
from kfp import dsl
from kfp.components import func_to_container_op, InputPath, OutputPath

kfp.__version__

'1.8.19'

In [15]:
import os

with open(os.environ['KF_PIPELINES_SA_TOKEN_PATH'], "r") as f:
    TOKEN = f.read()

In [3]:
endpoint = 'http://ml-pipeline.kubeflow:8888'
kfp_client = kfp.Client(host=endpoint, existing_token=TOKEN)

In [5]:
import torch
import torch.nn as nn
from torchvision import datasets, models, transforms, utils

torch.__version__

'1.8.1+cpu'

In [12]:
from minio import Minio

client = Minio(
    'minio.kubeflow:9000',
    access_key='minik8s',
    secret_key='minik8sio',
    secure=False,
)
client.bucket_exists("mnist")

True

In [14]:
transform = transforms.Compose([
    transforms.ToTensor(), # 轉為 Tensor
    transforms.Lambda(lambda x: x.repeat(3, 1, 1)), # 灰階轉為 RGB
])
mnist_train = datasets.MNIST(
    root='./data',       # 資料放置路徑
    train=True,          # 訓練資料集
    download=True,       # 自動下載
    transform=transform  # 轉換函數
)